In [ ]:
!pip install nltk

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import joblib
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer
from random import sample
import multiprocessing
from time import time
from gensim.models import Word2Vec

import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
all_tweets = joblib.load('./mps_2017_tweets.pckle')

all_sentences = []
 
def extract_sentences(mps_tweets_df):
    mps_tweets_df['author'] = mps_tweets_df['author'].apply(lambda x: x.lower())
    mps_tweets_df['author'] = mps_tweets_df['author'].apply(lambda x: x.split()[0])

    author_contents_mps_tweets_df = mps_tweets_df[['author', 'contents']].copy()

    dict_of_mps = {k: v for k, v in author_contents_mps_tweets_df.groupby('author')['contents']}
    new_dict_of_mps = {}

    for k, v in dict_of_mps.items():
        vals = []
        for val in v:
            tokeniser = TweetTokenizer()
            tokens = tokeniser.tokenize(val)
            filtered_sentence = [w for w in tokens if not w in stopwords.words()]
            vals.append(filtered_sentence)
        new_dict_of_mps[k] = vals

    return new_dict_of_mps

for dict_of_tweets in all_tweets[:-1]:
    all_sentences.append(extract_sentences(pd.DataFrame.from_dict(dict_of_tweets['posts'])))

In [ ]:
sentences_to_sample = []
for day in all_sentences:
    for sentence in day.values():
        sentences_to_sample += sentence
        
limit = len(sentences_to_sample)

for sentence in range(0, limit):
    temp = sample(sentences_to_sample[sentence], len(sentences_to_sample[sentence]))
    sentences_to_sample += temp

In [ ]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

w2v_model = Word2Vec(min_count=1,
                     window=2,
                     size=100,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

t = time()

w2v_model.build_vocab(sentences_to_sample, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()

w2v_model.train(sentences_to_sample, total_examples=w2v_model.corpus_count, epochs=50, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model.init_sims(replace=True)

w2v_model.save("w2v_model2018.model")